In [2]:
import sys
import psycopg2
import requests


In [3]:
table = 'record'


In [20]:
conn = psycopg2.connect("dbname='dbname' user='user' host='localhost' password='password'")

with conn.cursor() as cur:

    select_rows_stm = '''SELECT t1.record_id, t1.product_name, t1.downloaded
    FROM {table}_product_list t1
    LEFT OUTER JOIN {table}_contracts t2 ON t1.record_id = t2.record_id
    WHERE t2.record_id IS NULL'''.format(table=table)
    cur.execute(select_rows_stm)
    rows = cur.fetchall()

    for idx, row in enumerate(rows, start=1):
        row_id = row[0]
        text = row[1]
        downloaded = row[2]

        print(text)
        if downloaded is not True:
            r = requests.get(url='https://www.hlidacstatu.cz/api/v1/search',
                             params={'query': 'textSmlouvy:"{}"'.format(text)}, headers={'Authorization': 'Token ' + 'TOKEN'})
            data = r.json()
            print(len(data['items']))
            for item in data['items']:
                contract = {'record_id': row_id,
                            'sign_date': item['datumUzavreni'],
                            'payer': item['Platce']['ico'],
                            'supplier': item['Prijemce'][0]['ico'],
                            'description': item['predmet'],
                            'amount': item['CalculatedPriceWithVATinCZK']}
                insert_stm = '''INSERT INTO {table}_contracts (record_id, sign_date, payer, supplier, description, amount) 
                VALUES (%s, %s, %s, %s, %s, %s)'''.format(table=table)
                cur.execute(insert_stm, list(contract.values()))
            update_stm = '''UPDATE {table}_product_list SET downloaded=True WHERE record_id=%s'''.format(table=table)
            cur.execute(update_stm, [row_id])
    conn.commit()
        

senior developer
14
Acrobat 11 Pro CZ NEW GOV
0
AD 87
4
AD 8742
0
AD NOTAM
3
Adobe 10
4
GIS
40
Adobe 4
4
ADOBE LIVECYCLE
40
Adobe Photoshop
40
AFICIO MP C 3001 AD RICOH
0
AFICIO MP C 302 AD
0
iPad 2 35
0
iPhone 4 S
16
CS 715 MV
0
Aid Law 2010
0
Cisco IP 35
0
Cisco Nexus 7010
2
CKB AFCOS 2011
0
COBIT 5 Foundation
10
Code - 203370
0
Coloraction A 4
0
CorelDRAW Graphics Suite X 5 Win CZ
0
Crossbeam APM 4
0
CS 325 SMART A - polygrafie
0
Dell 682
0
Dell E 5430
0
Dell E 6410
1
Dell Latitude E 5430
0
Dell Latitude E 7440
0
Dell Optiplex 760 74
0
DFX 9000
26
Digicolor A 4
0
DUAL 1000
4
DVD DX 8000
0
DVR Intellex DVMS
5
E 4500
4
EB 100
40
EB 600
40
EBA - 2226 S
3
ECG MTM Goddess RSC 085 GW 8 S
0
ECG MTM VENUS RM 140 A + )
0
EHP - FMN 2 CZ 10
0
EMC NetWorker
40
EMC VMAX 10
0
ENVISHOP 2016
0
Alcatel Onetouch
3
ALI 1520
2
ALIGATOR R 10
0
Alto TW 300
0
Antitrust 2014
0
Apple iPhone 5S
12
ASSA ABLOY SGK
0
ASUS EN 210 / DI / 512 MD 2 Silent
0
Audi A6
40
AUDI A8
40
AutoCAD LT
40
BENQ MX 518
0
BENQ W 1